In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
urls = pd.read_csv('To Watch.csv')
urls

,href
0,https://myanimelist.net/anime/49303/Alice_to_T...
1,https://myanimelist.net/anime/49834/Boku_ga_Ai...
2,https://myanimelist.net/anime/49835/Kimi_wo_Ai...
3,https://myanimelist.net/anime/51307/Bokura_no_...
4,https://myanimelist.net/anime/52186/Kin_no_Kun...
5,https://myanimelist.net/anime/51116/Kagami_no_...
6,https://myanimelist.net/anime/51817/Watashi_no...
7,https://myanimelist.net/anime/486/Kino_no_Tabi...
8,https://myanimelist.net/anime/46095/Vivy__Fluo...
9,https://myanimelist.net/anime/12437/Yuri_Seiji...


In [3]:
url_string = urls['href']
print(url_string.head())

# set up output DF
processed = pd.DataFrame()
processed['Show Title'] = None
processed['Alt Show Title'] = None
processed['Blank Title'] = None
processed['Year'] = None
processed['Seasons'] = None
processed['Episodes'] = None
processed['Blank Episodes'] = None
processed['Duration'] = None
processed['Source Material'] = None
processed['Genres'] = None
processed['Blank Genres'] = None
processed['Blank Genres 2'] = None
processed['Studio'] = None
processed['Blank Studio'] = None
processed['Blank Studio 2'] = None
processed['Type']= None

# process each url and append extracted data to output DF
for i in range(len(url_string)):
    # generate page
    page = requests.get(url_string[i])
    print("Accessing: ", url_string[i], "\nStatus code: ", page.status_code)
    
    # parse page for data
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find(id="contentWrapper")
    
    # Show Title(s)
    show_title = container.find("h1", class_="title-name h1_bold_none")
    title_string = show_title.text.strip()
    alt_show_title = container.find("p", class_="title-english title-inherit")
    alt_title_string = ""
    if alt_show_title:
        alt_title_string = alt_show_title.text.strip()
    
    # Genres
    genres = container.find_all("span", itemprop="genre")
    genre_string = genres[0].text.strip()
    for genre in genres[1::]:
        genre_string += ", "
        genre_string += genre.text.strip()
        
    # Studios
    studios = container.find("span", class_="information studio author")
    studios = studios.find_all("a")
    studio_string = studios[0].text.strip()
    for studio in studios[1::]:
        studio_string += ", "
        studio_string += studio.text.strip()
    
    # Misc
    information = container.find_all("div", class_="spaceit_pad")
    information = list(information)
    for index in range(len(information)):
        information[index] = information[index].text.strip()
    type = ""
    aired = ""
    seasons = 1
    episodes = 1
    duration = ""
    source = ""
    for index in range(len(information)):
        # Type
        if information[index].lower().__contains__("type"):
            if information[index].lower().__contains__("movie"):
                type = "Movie"
            else:
                type = "Not Movie"
            continue
        
        # Episodes
        elif information[index].lower().__contains__("episodes"):
            episodes_string = information[index]
            episodes = int(episodes_string[-3::])
            continue
        
        # Aired
        elif information[index].lower().__contains__("aired:"):
            if type == 'Movie':
                aired = information[index][-4::]
            else:
                if information[index][14] == ",":
                    aired = information[index][16:21:]
                else:
                    aired = information[index][17:21:]
            continue
            
        # Source
        elif information[index].lower().__contains__("source:"):
            source = information[index].replace("Source:\n  ", "")
            if source.lower().__contains__("4-koma"):
                source = source.replace(" manga", "")
            if source.lower() != "manga" and source.lower() != "web manga" and source.lower() != "light novel" and source.lower() != "4-koma" and source.lower() != "visual novel" and source.lower() != "original":
                source = "Other"
            source = source.title()
            continue
                    
        # Duration
        elif information[index].lower().__contains__("duration:"):
            if type == "Movie":
                duration = information[index].replace("Duration:\n  ", "").replace(" hr.", ":")
                if duration.__contains__("min"):
                    duration = duration.replace(" min.", "")
                    if int(duration[3::]) < 10:
                        duration = duration.replace(":", ":0")
                    duration = duration.replace(" ", "")
                else:
                    duration += "00"
            else:
                duration = (information[index][12:14])
                if int(duration) <= 15:
                    type = "Short"
                else:
                    type = "TV"
            continue

    # append extracted data to output DF
    row = [[title_string, alt_title_string, "", aired, seasons, episodes, "", duration, source, genre_string, "", "", studio_string, "", "", type]]
    df = pd.DataFrame(row, columns=['Show Title', 'Alt Show Title', 'Blank Title', 'Year', 'Seasons', 'Episodes', 'Blank Episodes', 'Duration', 'Source Material', 'Genres', 'Blank Genres', 'Blank Genres 2', 'Studio', 'Blank Studio', 'Blank Studio 2', 'Type'])
    processed = pd.concat([processed, df], ignore_index=True)
processed.head() 

# write extracted data to output file

0    https://myanimelist.net/anime/49303/Alice_to_T...
1    https://myanimelist.net/anime/49834/Boku_ga_Ai...
2    https://myanimelist.net/anime/49835/Kimi_wo_Ai...
3    https://myanimelist.net/anime/51307/Bokura_no_...
4    https://myanimelist.net/anime/52186/Kin_no_Kun...
Name: href, dtype: object
Accessing:  https://myanimelist.net/anime/49303/Alice_to_Therese_no_Maboroshi_Koujou 
Status code:  200
Accessing:  https://myanimelist.net/anime/49834/Boku_ga_Aishita_Subete_no_Kimi_e 
Status code:  200
Accessing:  https://myanimelist.net/anime/49835/Kimi_wo_Aishita_Hitori_no_Boku_e 
Status code:  200
Accessing:  https://myanimelist.net/anime/51307/Bokura_no_Yoake 
Status code:  200
Accessing:  https://myanimelist.net/anime/52186/Kin_no_Kuni_Mizu_no_Kuni 
Status code:  200
Accessing:  https://myanimelist.net/anime/51116/Kagami_no_Kojou 
Status code:  200
Accessing:  https://myanimelist.net/anime/51817/Watashi_no_Yuri_wa_Oshigoto_desu 
Status code:  200
Accessing:  https://myanimelist.net/a

ValueError: invalid literal for int() with base 10: 'own'

In [ ]:
processed['More Info'] = urls
processed

In [ ]:
shows = processed[processed['Type'] == 'TV']
movies = processed[processed['Type'] == 'Movie']
shorts = processed[processed['Type'] == 'Short']
processed.shape, shows.shape, movies.shape, shorts.shape

In [ ]:
shows = shows.drop(columns=["Type"])

In [ ]:
shows

In [ ]:
movies = movies.drop(columns=["Seasons", "Episodes", "Blank Episodes", "Type"])

In [ ]:
movies

In [ ]:
shorts = shorts.drop(columns=["Type"])

In [ ]:
shorts

In [ ]:
shows.to_csv("results\shows.csv", index=False, sep='\t', header=False)
movies.to_csv("results\movies.csv", index=False, sep='\t', header=False)
shorts.to_csv("results\shorts.csv", index=False, sep='\t', header=False)